# OpenAI Function Calling In LangChain

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from typing import List
from pydantic import BaseModel, Field

In [ ]:
# This is a normal Python class:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [ ]:
# If we create an instance; its like any normall instance:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [ ]:
foo.name

In [ ]:
foo.age

In [ ]:
# Now using pydantic:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [ ]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [ ]:
# It can print out the whole object and its different elements due to pydantic:
foo_p

In [ ]:
foo_p.name

In [ ]:
# Now if we pass in a string under the age attribute when its clearly defined as an int type we will see a validation error:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

In [ ]:
# Another thing about pydantic is that we can nest data structures:
class Class(BaseModel):
    students: List[pUser]

In [ ]:
# We can create an object that has the exact same structure:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [ ]:
obj

# Using Pydantic to create OpenAI function definitions

In [ ]:
class WeatherSearch(BaseModel):
    # Docstring:
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [ ]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

In [ ]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [ ]:
weather_function

In [ ]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [ ]:
convert_pydantic_to_openai_function(WeatherSearch1)

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
model = ChatOpenAI()

In [ ]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

In [ ]:
model_with_function = model.bind(functions=[weather_function])

In [ ]:
model_with_function.invoke("what is the weather in SF?")

In [ ]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [ ]:
model_with_forced_function.invoke("what is the weather in sf?")

In [ ]:
model_with_forced_function.invoke("hi!")

# Using in a chain 
## We can use this model bound to function in a chain as we normally would 

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [ ]:
chain = prompt | model_with_function

In [ ]:
chain.invoke({"input": "what is the weather in sf?"})

# Using multiple functions
## EVen better, we can pass a set of function and let the LLM decide which to use based on the question context

In [ ]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [ ]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [ ]:
model_with_functions = model.bind(functions=functions)

In [ ]:
model_with_functions.invoke("what is the weather in sf?")

In [ ]:
model_with_functions.invoke("what are three songs by Eminem?")